<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [3]:
#ANSWER
print(f"Status Code: {response.status_code}")


Status Code: 200


In [4]:
#ANSWER
print(f"Response Text: {response.text}")

Response Text: {"message": "success", "timestamp": 1719097686, "iss_position": {"latitude": "28.7064", "longitude": "-177.6754"}}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [5]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [6]:
#ANSWER
for key, value in response.headers.items():
    print(f"{key}: {value}")

Date: Sat, 22 Jun 2024 23:08:07 GMT
Server: Apache
Vary: Origin,Access-Control-Request-Method,Access-Control-Request-Headers
Access-Control-Allow-Origin: *
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: 0
X-Frame-Options: DENY
Content-Type: application/json
Keep-Alive: timeout=5, max=100
Connection: Keep-Alive
Transfer-Encoding: chunked


Print the content of the response (the data that the server returned):

In [7]:
#ANSWER
print(response.json())

{'api_status': 'ALPHA', 'request_timestamp': '2024-06-22T23:08:07.898242031Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-06-22T19:03:26.193566191Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 16, 'passes': [{'start': '2024-06-23T04:01:12.882Z', 'tca': '2024-06-23T04:06:12.882Z', 'end': '2024-06-23T04:11:27.882Z', 'aos_azimuth': 342, 'los_azimuth': 119, 'max_elevation': 22.0}, {'start': '2024-06-23T05:37:37.882Z', 'tca': '2024-06-23T05:43:07.882Z', 'end': '2024-06-23T05:48:17.882Z', 'aos_azimuth': 291, 'los_azimuth': 141, 'max_elevation': 29.0}, {'start': '2024-06-23T07:16:47.882Z', 'tca': '2024-06-23T07:20:47.882Z', 'end': '2024-06-23T07:24:27.882Z', 'aos_azimuth': 242, 'los_azimuth': 153, 'max_elevation': 6.0}, {'start': '2024-06-23T08:56:17.882Z', 'tca': '2024-06-23T08:59:17.882Z', 'end': '2024-06-23T09:01:57.882Z', 'aos_azimuth': 208, 'los_azimuth': 145, 'max_elevation': 3.0}, {'start': '2024-06-23T10:33:07.882Z', 'tca': 

Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
#ANSWER
print("Content-Type:", response.headers.get("Content-Type"))
print(type(response.headers.get("Content-Type")))

Content-Type: application/json
<class 'str'>


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2024-06-22T23:08:07.898242031Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-06-22T19:03:26.193566191Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 16, 'passes': [{'start': '2024-06-23T04:01:12.882Z', 'tca': '2024-06-23T04:06:12.882Z', 'end': '2024-06-23T04:11:27.882Z', 'aos_azimuth': 342, 'los_azimuth': 119, 'max_elevation': 22.0}, {'start': '2024-06-23T05:37:37.882Z', 'tca': '2024-06-23T05:43:07.882Z', 'end': '2024-06-23T05:48:17.882Z', 'aos_azimuth': 291, 'los_azimuth': 141, 'max_elevation': 29.0}, {'start': '2024-06-23T07:16:47.882Z', 'tca': '2024-06-23T07:20:47.882Z', 'end': '2024-06-23T07:24:27.882Z', 'aos_azimuth': 242, 'los_azimuth': 153, 'max_elevation': 6.0}, {'start': '2024-06-23T08:56:17.882Z', 'tca': '2024-06-23T08:59:17.882Z', 'end': '2024-06-23T09:01:57.882Z', 'aos_azimuth': 208, 'los_azimuth': 145, 'max_elevation': 3.0}, {'start': '2024-06-23T10:33:07.882Z', 'tca': 

What kind of object did this give us?

In [11]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [16]:
#ANSWER:
# Extracting the passes into a separate list
passes_list = overheads['passes']

# Printing the passes list
print(passes_list)

[{'start': '2024-06-23T04:01:12.882Z', 'tca': '2024-06-23T04:06:12.882Z', 'end': '2024-06-23T04:11:27.882Z', 'aos_azimuth': 342, 'los_azimuth': 119, 'max_elevation': 22.0}, {'start': '2024-06-23T05:37:37.882Z', 'tca': '2024-06-23T05:43:07.882Z', 'end': '2024-06-23T05:48:17.882Z', 'aos_azimuth': 291, 'los_azimuth': 141, 'max_elevation': 29.0}, {'start': '2024-06-23T07:16:47.882Z', 'tca': '2024-06-23T07:20:47.882Z', 'end': '2024-06-23T07:24:27.882Z', 'aos_azimuth': 242, 'los_azimuth': 153, 'max_elevation': 6.0}, {'start': '2024-06-23T08:56:17.882Z', 'tca': '2024-06-23T08:59:17.882Z', 'end': '2024-06-23T09:01:57.882Z', 'aos_azimuth': 208, 'los_azimuth': 145, 'max_elevation': 3.0}, {'start': '2024-06-23T10:33:07.882Z', 'tca': '2024-06-23T10:37:37.882Z', 'end': '2024-06-23T10:41:42.882Z', 'aos_azimuth': 208, 'los_azimuth': 104, 'max_elevation': 9.0}, {'start': '2024-06-23T12:09:22.882Z', 'tca': '2024-06-23T12:14:52.882Z', 'end': '2024-06-23T12:20:17.882Z', 'aos_azimuth': 222, 'los_azimuth':

Now extract the `start` strings into an array called `srisetimes`:

In [17]:
#ANSWER:
# Extracting the start strings into an array
srisetimes = [pass_info['start'] for pass_info in overheads['passes']]

# Printing the srisetimes array
print(srisetimes)

['2024-06-23T04:01:12.882Z', '2024-06-23T05:37:37.882Z', '2024-06-23T07:16:47.882Z', '2024-06-23T08:56:17.882Z', '2024-06-23T10:33:07.882Z', '2024-06-23T12:09:22.882Z', '2024-06-23T13:46:47.882Z', '2024-06-24T03:13:52.882Z']


These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [19]:


# Converting start strings to datetime objects
risetimes = [datetime.fromisoformat(start_time.replace('Z', '+00:00')) for start_time in srisetimes]

# Printing the datetime array
print(risetimes)

[datetime.datetime(2024, 6, 23, 4, 1, 12, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 5, 37, 37, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 7, 16, 47, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 8, 56, 17, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 10, 33, 7, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 12, 9, 22, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 23, 13, 46, 47, 882000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 6, 24, 3, 13, 52, 882000, tzinfo=datetime.timezone.utc)]


Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [20]:
#ANSWER:
# Formatting datetime objects to a human-readable string
srisetimes_formatted = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in risetimes]

# Printing the formatted datetime array
print(srisetimes_formatted)

['2024-06-23 04:01:12', '2024-06-23 05:37:37', '2024-06-23 07:16:47', '2024-06-23 08:56:17', '2024-06-23 10:33:07', '2024-06-23 12:09:22', '2024-06-23 13:46:47', '2024-06-24 03:13:52']


Finally, here is an endpoint that tells us who is on board:

In [26]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [27]:
#ANSWER:
# Extracting the number of astronauts
actual_output = response.json()
number_of_astronauts = actual_output['number']

# Extracting the names of the astronauts
astronaut_names = [person['name'] for person in actual_output['people']]

# Printing the results
print(f"Number of astronauts: {number_of_astronauts}")
print("Names of astronauts:")
for name in astronaut_names:
    print(name)


Number of astronauts: 12
Names of astronauts:
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [36]:
#ANSWER:
def handleResponse(response, verbose=False):
    '''
    Returns a tuple with a Boolean value and the status code.

    If the status code is 200, returns (False, status_code).
    Otherwise, returns (True, status_code).

    If verbose is True, it will also print the status code and the message.
    '''
    status_code = response.status_code
    
    if status_code == 200:
        result = (False, status_code)
    else:
        result = (True, status_code)

    if verbose:
        message = response.json().get('message', 'No message provided') if response.headers.get('Content-Type') == 'application/json' else 'Response is not JSON'
        print(f"Status Code: {status_code}, Message: {message}")
    
    return result

2. Test your response handler on some correct and incorrect API calls.

In [37]:
# First API call
response = requests.get("http://api.open-notify.org/astros.json")

if handleResponse(response, False)[0]:
    print('API call failed. Resolve issue before continuing!')

# Second API call
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)

Status Code: 200, Message: success


(False, 200)

>

>

>



---



---



> > > > > > > > > © 2024 Institute of Data


---



---



